In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta, date, time
import urllib3
import logging
from utils.logging_setup import logging_setup
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
version = 1
token = 'HGEGOBANHUEY'
esios_key = "a27b325f0fbd4d8ffa46b6f2a015595b87d3bb247d28feed89efbb3bc650376f"
indicador = 600

In [3]:
# Parameters
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
log_name = "markets_exe"
festivos_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/festivos"
periodos_cal_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/calendario_periodos.xlsx"
futures_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/futuros_elec.parquet"
spot_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_elec.parquet"
spot_formated_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_h_periodos.pkl"


In [ ]:
logging_setup(log_name=log_name, config_path=config_path)

In [6]:
from shared.esios_api import EsiosAPI

In [7]:
api_esios = EsiosAPI(key=esios_key,
                               df_prices=pd.DataFrame()
                               )

In [8]:
# Ejemplo de  descarga de cotizaciones de futuros para un proveedor de datos en concretoa través de API

# if os.path.exists(futures_path):

#     precios = pd.read_parquet(futures_path)
#     precios['Day'] = pd.to_datetime(precios['Day'])
#     start_date = ((precios['Day'].max()) + timedelta(days=1))
#     precios['Day'] = precios['Day'].dt.date

# else:
#     precios = pd.DataFrame()
#     start_date = datetime(2017,1,1)

# end_date =  datetime.now() - timedelta(days=1)
# date_range = (end_date - start_date + timedelta(days=1)).days

# start = start_date.date()
# end = end_date.date()
# print(date_range)
# print(start)
# print(end)

# if start_date <= end_date:    
#     df_precios = api.fetch_full_data_range(start, end, endpoint='/')
# else:
#     df_precios = pd.DataFrame()

# if not df_precios.empty:
#     # df_precios = df_precios[(df_precios['Type'] == 'FT') & (df_precios['Profile'] == 'BL')]
#     df_precios['Day'] = pd.to_datetime(df_precios['Day'])
#     df_precios['Day'] = df_precios['Day'].apply(lambda x: x.date())
#     df_precios = df_precios[['Day', 'Contract', 'Delivery', 'Year', 'Type','FirstTradingDay', 'LastTradingDay', 'FirstDeliveryDay', 'LastDeliveryDay', 'Hours', 'SettlementPrice']]
#     futuros_elec = pd.concat([precios, df_precios], ignore_index=True)
#     futuros_elec.to_parquet(futures_path,engine='pyarrow',index=False)
# else:
#     futuros_elec = precios


In [9]:
if os.path.exists(spot_path):
    spot_prices = pd.read_parquet(spot_path)
    start_date = ((spot_prices['datetime'].max()) + timedelta(days=1))
    start_date = start_date.date()
else:
    spot_prices = pd.DataFrame()
    start_date = date(2014,1,1)
    
start_date = datetime.combine(start_date, time(0,0,0))
end_date =  datetime.now() # datetime(2015,12,1)
end_date =  end_date.date()
end_date = datetime.combine(end_date, time(23,0,0))
date_range = (end_date - start_date + timedelta(days=1)).days
logging.info(date_range)
logging.info(start_date)
logging.info(end_date)

if start_date <= end_date:
    print('API request triggered')
    df_response = api_esios.get_raw_data(
                                        indicator=600, 
                                        start_date=start_date, 
                                        end_date=end_date
                                        )
    df_response =  api_esios.df_response
else:
    df_response = pd.DataFrame()
    logging.info('Precios actualizados')   


if not df_response.empty:
    if not spot_prices.empty:
        spot_prices = pd.concat([spot_prices, df_response], ignore_index=True)
        spot_prices.to_parquet(spot_path, engine='pyarrow',index=False)
        print('Spot prices updated, merged and correctly exported to parquet')
    else:
        spot_prices = df_response
        spot_prices.to_parquet(spot_path, engine='pyarrow',index=False)        
        print('Spot prices correctly exported to parquet\n')
else:
    logging.info('No action necesary\n')

0
2025-08-17 00:00:00
2025-08-16 23:00:00
Precios actualizados
No action necesary


In [10]:
from shared.elec_price_op import SpotElec

In [11]:
HourSpot = SpotElec(spot_prices)
HourSpot.format().apply_periodos(
                            periodos_path=periodos_cal_path,
                            festivos_path=festivos_path
                                );
HourSpot.df_w_periodos.to_pickle(spot_formated_path)